In [ ]:
import sys
import re
import time
import datetime
import pandas as pd
import numpy as np
import func
# inline plot
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("data/merged_concat_final.csv",sep='\t',error_bad_lines=False)
del df['Unnamed: 0']
print df.shape

In [ ]:
df[['del_min','del_med','del_maj','del_cat']].sum()

In [ ]:
df.set_index('timestamp')
df['timestamp'] = pd.to_datetime(df['timestamp'],format="%Y-%m-%d %H:%M:%S")

In [ ]:
def delay_to_ordinal(r):
    v = 0 #no delay
    v = 1 if r['del_min'] == 1 else v
    v = 2 if r['del_med'] == 1 else v
    v = 3 if r['del_maj'] == 1 else v
    v = 4 if r['del_cat'] == 1 else v
    return v

df['ord_del'] = df.apply(lambda x:delay_to_ordinal(x),axis=1)

In [ ]:
def days_to_ordinal(r):
    v = 0 # Sunday
    v = 1 if r['d_monday'] == 1 else v
    v = 2 if r['d_tuesday'] == 1 else v
    v = 3 if r['d_wednesday'] == 1 else v
    v = 4 if r['d_thursday'] == 1 else v
    v = 5 if r['d_friday'] == 1 else v
    v = 5 if r['d_saturday'] == 1 else v
    return v

df['ord_weekdays'] = df.apply(lambda x:days_to_ordinal(x),axis=1)

In [ ]:
only_delay = df[(df['is_delay']==1)]

In [ ]:
df.plot.scatter(x=df['timestamp'],y=df['ord_del'],figsize=[15,6], alpha='0.2')

In [ ]:
# only_delay[['del_min','del_med','del_maj','del_cat','is_bullet','is_limited']].plot.hist(color='k',alpha=0.5,stacked=True,bins=4,figsize=[12,6])

In [ ]:
df.dtypes

In [ ]:
print (df['del_cat'].sum()/float(df['is_delay'].sum()))*100 , '%'

In [ ]:
df['timestamp'].dtype

In [ ]:
# Non Delay Specific
features = df.columns.values
X = df[['temp','precipiation',
        'visability','windspeed','humidity','cloudcover',
        'is_bullet','is_limited','t_northbound',
       'd_monday','d_tuesday','d_wednesday','d_thursday','d_friday','d_saturday']]
# del X['is_delay']
# del X['tweet_id']
# X['timestamp'] = X['timestamp'].apply(lambda x: (np.datetime64(x).astype('uint64') / 1e6).astype('uint32'))
y = df['del_ord']

In [ ]:
# X['timestamp'] = X['timestamp'].apply(lambda x:int(x))
# X['stop_pa'] = X['stop_pa'].apply(lambda x:int(x))
# X['train_id'] = X['train_id'].apply(lambda x:int(x))
X['t_northbound'] = X['t_northbound'].apply(lambda x:int(x))

In [ ]:
X['cloudcover'] = X['cloudcover'].fillna(X['cloudcover'].mean())

In [ ]:
X.isnull().sum()

In [ ]:
df.plot.scatter(x='timestamp',y='del_ord',figsize=[15,5])

In [ ]:
X_y = only_delay[['is_delay','ord_del','temp','precipiation',
        'visability','windspeed','humidity','cloudcover',
        'is_bullet','is_limited','t_northbound',
       'd_monday','d_tuesday','d_wednesday','d_thursday','d_friday','d_saturday']]
cr = X_y.corr()
np.round(cr, 4)
# 

In [ ]:
X_y.sum()

### Find the Principal Components

In [ ]:
X = only_delay[['temp','precipiation',
        'visability','windspeed','humidity','cloudcover',
        'is_bullet','is_limited','t_northbound',
       'd_monday','d_tuesday','d_wednesday','d_thursday','d_friday','d_saturday']]

In [ ]:
from sklearn.decomposition import PCA
clf = PCA(.99)
X_trans = clf.fit_transform(X)
X_trans.shape

In [ ]:

plt.scatter(X_trans[:, 0], X_trans[:, 1],c=y, alpha=0.2)
plt.colorbar();

In [ ]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm.fit(X_trans,y)

In [ ]:
print(lm.intercept_)
print(lm.coef_)

In [ ]:
from sklearn import feature_selection
pvals = feature_selection.f_regression(X,y)[1] 
zip(X.columns.values,np.round(pvals,4))


In [ ]:
from sklearn.cross_validation import cross_val_score
print(cross_val_score(lm,X_trans,y,cv=10).mean()) 
MisClassificationError = 1 - (cross_val_score(lm,X_trans,y,cv=10).mean())
print(MisClassificationError)

### Seeing if I can get anything interesting out of KNN given above

In [ ]:
print df['windspeed'].max()
print df['windspeed'].min()

In [ ]:
df['windspeed_st'] = df['windspeed'].apply(lambda x:x/15.0) # Ballparking 

In [ ]:

X_reg = df[['precipiation','d_friday','t_northbound','windspeed_st']]
y_reg = df['ord_del']

In [ ]:
from sklearn import cross_validation
from sklearn import neighbors, metrics
kf = cross_validation.KFold(len(X_reg), n_folds = 10, shuffle = True) #10 fold CV
Score_KNN_CV = []
RangeOfK = range(1,20) 
scores = []
for k in RangeOfK:
    knn = neighbors.KNeighborsClassifier(n_neighbors=k, weights='uniform')
    scores = []
    for train_index, test_index in kf:        
        knn.fit(X_reg.iloc[train_index], y_reg.iloc[train_index])
        scores.append(knn.score(X_reg.iloc[test_index],y_reg.iloc[test_index]))
    Score_KNN_CV.append(np.mean(scores))

Score_KNN_CV_df = pd.DataFrame({'Score_KNN_CV': Score_KNN_CV ,'K': RangeOfK })
Score_KNN_CV_df.plot(x = 'K',y = 'Score_KNN_CV',figsize=[15,5])

### Cross Validation and Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score
RFClass = RandomForestClassifier(n_estimators = 10000, 
                                 max_features = 4, # You can set it to a number or 'sqrt', 'log2', etc
                                 min_samples_leaf = 5,
                                 oob_score = True)
RFClass.fit(X,y)
print(RFClass.oob_score_)
scores = cross_val_score(RFClass, X, y, cv=10)
print(scores.mean())
#out of bag error = 25% , CV_error is 35%

In [ ]:
RFClass.fit(X,y)
ImportanceDataFrame = pd.DataFrame({'feature':X.columns.values, 'importance':RFClass.feature_importances_})
ImportanceDataFrame.sort_values(by = ['importance'],ascending = 0)

In [ ]:
Depth_Choice_df = pd.DataFrame({'cv_scores': score,'Number of Features': Features})
Depth_Choice_df.plot(x ='Number of Features',y = 'cv_scores')